# A1111 Model Installer — SD 1.5 (UI)

Install **Stable Diffusion 1.5** models and common assets into the WebUI data directory.

**Folders** (under `WEBUI_ROOT`, default `/workspace/a1111`):
- **Checkpoints** → `models/Stable-diffusion/`
- **VAE** → `models/VAE/`
- **ControlNet (SD1.5)** → `models/ControlNet/`
- **LoRA** → `models/Lora/`
- **Embeddings** → `embeddings/`

Re-run any time — existing files are skipped.

> Optional tokens: set `HF_TOKEN` (gated HF) and/or `CIVITAI_TOKEN` (private/authorized Civitai). Notebook still works without them.

In [ ]:
from pathlib import Pathimport osROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))CKPT_DIR   = ROOT / "models/Stable-diffusion"VAE_DIR    = ROOT / "models/VAE"CN_DIR     = ROOT / "models/ControlNet"LORA_DIR   = ROOT / "models/Lora"EMB_DIR    = ROOT / "embeddings"for d in (CKPT_DIR, VAE_DIR, CN_DIR, LORA_DIR, EMB_DIR):    d.mkdir(parents=True, exist_ok=True)print("Data root:", ROOT)print("↳ Checkpoints:", CKPT_DIR)print("↳ VAE:", VAE_DIR)print("↳ ControlNet:", CN_DIR)print("↳ LoRA:", LORA_DIR)print("↳ Embeddings:", EMB_DIR)

## Helpers (progress + smart filenames)
Accepts **Hugging Face** (repo + filename) or **direct URLs** (e.g., Civitai).

In [ ]:
import os, re, shutil, requestsfrom urllib.parse import urlparsefrom tqdm import tqdmfrom huggingface_hub import hf_hub_downloadfrom pathlib import Pathdef human(n: int) -> str:    for u in ("B","KB","MB","GB"):        if n < 1024: return f"{n:.1f} {u}"        n /= 1024    return f"{n:.1f} TB"def _save_stream_to(dest: Path, r: requests.Response) -> Path:    dest.parent.mkdir(parents=True, exist_ok=True)    if dest.exists():        print("✔ Exists:", dest); return dest    total = int(r.headers.get("Content-Length", 0))    tmp = dest.with_suffix(dest.suffix + ".part")    with open(tmp, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=dest.name, ncols=80) as bar:        for chunk in r.iter_content(chunk_size=1024*1024):            if chunk:                f.write(chunk); bar.update(len(chunk))    tmp.replace(dest)    print(f"✔ Saved: {dest} ({human(dest.stat().st_size)})")    return destdef _filename_from_headers_or_url(r: requests.Response, url: str, default_ext: str = ".safetensors") -> str:    cd = r.headers.get("Content-Disposition", "")    m = re.search(r'filename="?([^";]+)"?', cd)    if m:        return m.group(1)    name = url.split("/")[-1].split("?", 1)[0]    if "." not in name:        name += default_ext    return namedef download_direct(url: str, dest_dir: Path, dest_name: str|None=None) -> Path:    headers = {"User-Agent": "sd-webui-hub-notebook/1.0"}    netloc = urlparse(url).netloc.lower()    civitai_token = os.environ.get("CIVITAI_TOKEN")    if "civitai.com" in netloc and civitai_token:        headers["Authorization"] = f"Bearer {civitai_token}"    r = requests.get(url, stream=True, allow_redirects=True, headers=headers)    r.raise_for_status()    filename = dest_name or _filename_from_headers_or_url(r, url)    return _save_stream_to(dest_dir/filename, r)def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None) -> Path:    dest = dest_dir / (dest_name or filename)    if dest.exists():        print("✔ Exists:", dest); return dest    print(f"→ HF: {repo_id}/{filename}")    path = hf_hub_download(        repo_id=repo_id,        filename=filename,        token=os.environ.get("HF_TOKEN"),        resume_download=True    )    shutil.copy2(path, dest)    print(f"✔ HF saved: {dest} ({human(dest.stat().st_size)})")    return destdef fetch(item: dict, dest_dir: Path) -> None:    if "hf" in item:        repo, fname = item["hf"]["repo"], item["hf"]["filename"]        download_hf(repo, fname, dest_dir, item.get("dest_name"))    elif "url" in item:        download_direct(item["url"], dest_dir, item.get("dest_name"))    else:        raise ValueError("Item must contain 'hf' or 'url'.")def download_extras(lines: str, dest_dir: Path) -> None:    for url in [u.strip() for u in lines.splitlines() if u.strip()]:        try:            download_direct(url, dest_dir)        except Exception as e:            print("✖", url, "→", e)

## Checkpoints (paste URLs)Paste one per line (Civitai or HF direct links). Files go to `models/Stable-diffusion/`.

In [ ]:
import ipywidgets as Wfrom IPython.display import display, Markdownckpt_text = W.Textarea(    value="",    placeholder="Paste SD1.5 checkpoint URLs (.safetensors/.ckpt), one per line.",    layout=W.Layout(width="100%", height="120px"))ckpt_btn = W.Button(description="Download Checkpoints", button_style="success", icon="download")ckpt_out = W.Output()def dl_ckpts(_):    ckpt_out.clear_output()    with ckpt_out:        urls = [u.strip() for u in ckpt_text.value.splitlines() if u.strip()]        for url in urls:            try:                name = url.split("/")[-1].split("?", 1)[0]                if not any(name.endswith(ext) for ext in (".safetensors", ".ckpt")):                    name += ".safetensors"                download_direct(url, CKPT_DIR, dest_name=name)            except Exception as e:                print("✖", url, "→", e)        print("✔ Checkpoints done.")ckpt_btn.on_click(dl_ckpts)display(ckpt_text, ckpt_btn, ckpt_out)

## VAE (choose one or both)Downloads to `models/VAE/`.

In [ ]:
vae_items = [  {"name": "SD1.5 MSE VAE (stabilityai)", "hf": {"repo": "stabilityai/sd-vae-ft-mse", "filename": "vae-ft-mse-840000-ema-pruned.safetensors"}},  {"name": "SD1.5 EMA VAE (stabilityai)", "hf": {"repo": "stabilityai/sd-vae-ft-ema", "filename": "vae-ft-ema-560000-ema-pruned.safetensors"}}]vae_checks = [W.Checkbox(description=i["name"], value=False) for i in vae_items]vae_btn = W.Button(description="Download VAE", icon="download")vae_out = W.Output()def dl_vae(_):    vae_out.clear_output()    with vae_out:        for cb, item in zip(vae_checks, vae_items):            if cb.value:                fetch(item, VAE_DIR)        print("✔ VAE done.")vae_btn.on_click(dl_vae)for w in vae_checks: display(w)display(vae_btn, vae_out)

## ControlNet (SD 1.5) — pre-checked common onesDownloads to `models/ControlNet/`.

In [ ]:
cn_items = [  {"name": "Canny",     "hf": {"repo": "lllyasviel/control_v11p_sd15_canny",     "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "control_v11p_sd15_canny.safetensors"},  {"name": "SoftEdge",  "hf": {"repo": "lllyasviel/control_v11p_sd15_softedge",  "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "control_v11p_sd15_softedge.safetensors"},  {"name": "OpenPose",  "hf": {"repo": "lllyasviel/control_v11p_sd15_openpose",  "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "control_v11p_sd15_openpose.safetensors"},  {"name": "Depth",     "hf": {"repo": "lllyasviel/control_v11p_sd15_depth",     "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "control_v11p_sd15_depth.safetensors"},  {"name": "LineArt",   "hf": {"repo": "lllyasviel/control_v11p_sd15_lineart",   "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "control_v11p_sd15_lineart.safetensors"},  {"name": "NormalBae", "hf": {"repo": "lllyasviel/control_v11p_sd15_normalbae", "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "control_v11p_sd15_normalbae.safetensors"}]pre = {"Canny","SoftEdge","OpenPose"}cn_checks = [W.Checkbox(description=i["name"], value=(i["name"] in pre)) for i in cn_items]cn_extra = W.Textarea(placeholder="Extra ControlNet model URLs (HF or direct), one per line", layout=W.Layout(width="100%", height="80px"))cn_btn = W.Button(description="Download ControlNet", icon="download")cn_out = W.Output()def dl_cn(_):    cn_out.clear_output()    with cn_out:        for cb, item in zip(cn_checks, cn_items):            if cb.value:                # Use dest_name if provided                if "dest_name" in item:                    fetch({**item, "hf": item.get("hf")}, CN_DIR)                else:                    fetch(item, CN_DIR)        download_extras(cn_extra.value, CN_DIR)        print("✔ ControlNet done.")cn_btn.on_click(dl_cn)for w in cn_checks: display(w)display(W.HTML("<b>Extra URLs</b>"), cn_extra, cn_btn, cn_out)

## LoRA (pre-checked examples) — goes to `models/Lora/`

In [ ]:
lora_items = [  {"name": "RealisticVision+ Enhancer (LoRA)", "url": "https://civitai.com/api/download/models/712234?type=Model&format=SafeTensor", "dest_name": "RealisticVisionEnhancer.safetensors"},  {"name": "Detail Tweaker (LoRA)",            "url": "https://civitai.com/api/download/models/715574?type=Model&format=SafeTensor", "dest_name": "DetailTweaker.safetensors"}]lora_pre = {"RealisticVision+ Enhancer (LoRA)", "Detail Tweaker (LoRA)"}lora_checks = [W.Checkbox(description=i["name"], value=(i["name"] in lora_pre)) for i in lora_items]lora_extra = W.Textarea(placeholder="Extra LoRA URLs (.safetensors), one per line", layout=W.Layout(width="100%", height="80px"))lora_btn = W.Button(description="Download LoRA", icon="download")lora_out = W.Output()def dl_lora(_):    lora_out.clear_output()    with lora_out:        for cb, item in zip(lora_checks, lora_items):            if cb.value:                fetch(item, LORA_DIR)        download_extras(lora_extra.value, LORA_DIR)        print("✔ LoRA done.")lora_btn.on_click(dl_lora)for w in lora_checks: display(w)display(W.HTML("<b>Extra URLs</b>"), lora_extra, lora_btn, lora_out)

## Embeddings (Textual Inversion) — `embeddings/`

In [ ]:
emb_extra = W.Textarea(placeholder="Embeddings (.pt/.bin/.safetensors) URLs — one per line", layout=W.Layout(width="100%", height="80px"))emb_btn = W.Button(description="Download Embeddings", icon="download")emb_out = W.Output()def dl_emb(_):    emb_out.clear_output()    with emb_out:        download_extras(emb_extra.value, EMB_DIR)        print("✔ Embeddings done.")emb_btn.on_click(dl_emb)display(emb_extra, emb_btn, emb_out)

## Restart WebUI (after downloads)
Restarts A1111 (under supervisord). If the control socket isn't available, we kill `launch.py` and let Supervisor auto-restart.

In [ ]:
import time, subprocess, shlexprint("Attempting to restart the A1111 WebUI service via supervisord …")try:    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)    if out.returncode == 0 and "a1111:" in (out.stdout or ""):        print((out.stdout or out.stderr).strip() or "✔ Restarted via supervisorctl")    else:        print("Supervisorctl unavailable, forcing process restart…")        subprocess.run("pkill -f 'launch.py'", shell=True)        time.sleep(5)        print("✔ WebUI process terminated; Supervisor will auto-restart it")    print("Wait ~15 seconds, then refresh the WebUI tab (7860).")except Exception as e:    print("⚠️ Restart attempt failed:", e)    print("Manual command:")    print("supervisorctl -c /etc/supervisord.conf restart a1111")